In [1]:
from keras.datasets import mnist
from keras.utils import to_categorical

from keras.models import Sequential
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPool2D
from keras.layers.core import Dense, Dropout, Flatten
from keras.optimizers import Adam

/home/tatsuki/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/tatsuki/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/tatsuki/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/tatsuki/anaconda3/lib/python3.6/site-p

In [2]:
(X_train, y_train),(X_test, y_test) = mnist.load_data()

### Normalization (Required for image data)

In [3]:
X_train = X_train.reshape(X_train.shape[0], 28, 28, 1)
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

X_train /= 255.0
X_test  /= 255.0

### conversion

In [4]:
y_train = to_categorical(y_train)
y_test  = to_categorical(y_test)

In [5]:
X_test.shape

(10000, 28, 28, 1)

Tensorboard

In [ ]:
import keras.callbacks
import keras.backend.tensorflow_backend as KTF
import tensorflow as tf

old_session = KTF.get_session()
session = tf.Session('')
KTF.set_session(session)
KTF.set_learning_phase(1)

### CNN Model

In [6]:
def build_model():
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3,3), input_shape=(28, 28, 1), activation='relu'))
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Conv2D(32, kernel_size=(3,3), activation='relu'))
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(10, activation='softmax'))
    
    return model

In [7]:
model = build_model()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 32)        9248      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 32)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 800)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               205056    
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
__________

In [8]:
model.compile(optimizer='Adam',
              loss='categorical_crossentropy',
             metrics=['accuracy'])

Tensorboard

In [10]:
import keras.callbacks
tb_cb = keras.callbacks.TensorBoard(log_dir='./tflog', histogram_freq=1)

learning

In [12]:
history = model.fit(X_train, y_train,
                    validation_data=(X_test, y_test),
                    batch_size=100,
                    epochs=5,
                    verbose=1,
                    callbacks=[tb_cb])

Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 35s 575us/step - loss: 0.2036 - acc: 0.9382 - val_loss: 0.0596 - val_acc: 0.9813
Epoch 2/5
60000/60000 [==============================] - 36s 597us/step - loss: 0.0576 - acc: 0.9822 - val_loss: 0.0374 - val_acc: 0.9879
Epoch 3/5
60000/60000 [==============================] - 37s 616us/step - loss: 0.0389 - acc: 0.9883 - val_loss: 0.0354 - val_acc: 0.9871
Epoch 4/5
60000/60000 [==============================] - 36s 602us/step - loss: 0.0295 - acc: 0.9905 - val_loss: 0.0267 - val_acc: 0.9912
Epoch 5/5
60000/60000 [==============================] - 35s 592us/step - loss: 0.0225 - acc: 0.9928 - val_loss: 0.0280 - val_acc: 0.9898


In [13]:
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss: {}'.format(score[0]))
print('Test accuracy: {}'.format(score[1]))

Test loss: 0.02799537419131957
Test accuracy: 0.9898


### Tensorboard

$ tensorboard --logdir=./tflog

localhost:6006